In [1]:
import pandas as pd
import numpy as np

In [5]:
train_df=pd.read_csv("/kaggle/input/burnout-datathon-ieeecsmuj/train.csv")
test_df=pd.read_csv("/kaggle/input/burnout-datathon-ieeecsmuj/test.csv")
val_df=pd.read_csv("/kaggle/input/burnout-datathon-ieeecsmuj/val.csv")

In [6]:
print(train_df.shape,test_df.shape,val_df.shape)

(1914056, 45) (546874, 44) (273437, 45)


In [7]:
train_df.head()

,Unique ID,Rider_ID,category_x,Circuit_Length_km,Laps,Grid_Position,Avg_Speed_kmh,Track_Condition,Humidity_%,Tire_Compound_Front,...,air,ground,starts,finishes,with_points,podiums,wins,min_year,max_year,years_active
0,1894944,2659,Moto2,4.874,22,17,264.66,Wet,61,Hard,...,23,35,53,45,41,4,0,2018,2021,4
1,23438,5205,Moto2,3.875,24,7,177.56,Wet,77,Soft,...,12,12,27,27,22,2,1,1975,1983,8
2,939678,7392,Moto3,5.647,25,5,317.74,Dry,87,Soft,...,22,23,45,43,10,0,0,1982,1989,8
3,1196312,7894,Moto3,4.810,19,3,321.82,Wet,43,Soft,...,23,35,192,172,155,16,9,1994,2009,16
4,1033899,6163,MotoGP,5.809,25,21,239.92,Wet,47,Hard,...,22,31,175,146,132,29,17,2011,2021,11


In [17]:
train_df.nunique()

Unique ID                          2187493
Rider_ID                              8999
category_x                               3
Circuit_Length_km                     2401
Laps                                     8
Grid_Position                           22
Avg_Speed_kmh                        19961
Track_Condition                          2
Humidity_%                              60
Tire_Compound_Front                      3
Tire_Compound_Rear                       3
Penalty                                  6
Championship_Points                    350
Championship_Position                   24
Session                                  7
year_x                                  73
sequence                                19
rider                                 2695
team                                   970
bike                                   301
position                                43
points                                  27
shortname                               53
circuit_nam

In [9]:
import pandas as pd

# Assuming train_df and val_df are already defined
train_df = pd.concat([train_df, val_df], ignore_index=True)


In [10]:
print(train_df.shape,test_df.shape,val_df.shape)

(2187493, 45) (546874, 44) (273437, 45)


In [11]:
train_df.nunique()

Unique ID                          2187493
Rider_ID                              8999
category_x                               3
Circuit_Length_km                     2401
Laps                                     8
Grid_Position                           22
Avg_Speed_kmh                        19961
Track_Condition                          2
Humidity_%                              60
Tire_Compound_Front                      3
Tire_Compound_Rear                       3
Penalty                                  5
Championship_Points                    350
Championship_Position                   24
Session                                  7
year_x                                  73
sequence                                19
rider                                 2695
team                                   970
bike                                   301
position                                43
points                                  27
shortname                               53
circuit_nam

In [12]:
train_df.dtypes

Unique ID                            int64
Rider_ID                             int64
category_x                          object
Circuit_Length_km                  float64
Laps                                 int64
Grid_Position                        int64
Avg_Speed_kmh                      float64
Track_Condition                     object
Humidity_%                           int64
Tire_Compound_Front                 object
Tire_Compound_Rear                  object
Penalty                             object
Championship_Points                  int64
Championship_Position                int64
Session                             object
year_x                               int64
sequence                             int64
rider                                int64
team                                 int64
bike                                 int64
position                             int64
points                             float64
shortname                           object
circuit_nam

In [13]:
train_df.isna().sum()

Unique ID                               0
Rider_ID                                0
category_x                              0
Circuit_Length_km                       0
Laps                                    0
Grid_Position                           0
Avg_Speed_kmh                           0
Track_Condition                         0
Humidity_%                              0
Tire_Compound_Front                     0
Tire_Compound_Rear                      0
Penalty                            366912
Championship_Points                     0
Championship_Position                   0
Session                                 0
year_x                                  0
sequence                                0
rider                                   0
team                                    0
bike                                    0
position                                0
points                                  0
shortname                               0
circuit_name                      

In [14]:
train_df=train_df.fillna(0)

In [15]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from catboost import CatBoostRegressor
import time

# Target column
target_col = 'Lap_Time_Seconds'

# Drop rows with missing target
train_df = train_df.dropna(subset=[target_col])

# Split features and target
X = train_df.drop(columns=[target_col])
y = train_df[target_col]

# Split into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Identify categorical features (CatBoost handles them natively)
cat_features = X.select_dtypes(include=['object', 'category']).columns.tolist()

# Optional: Fill missing values
X_train = X_train.fillna(-999)
X_val = X_val.fillna(-999)

# Initialize CatBoost Regressor
model = CatBoostRegressor(
    iterations=1000,
    learning_rate=0.05,
    depth=6,
    eval_metric='RMSE',
    random_seed=42,
    verbose=100
)

# Record training start time
start_time = time.time()

# Train model
model.fit(X_train, y_train, cat_features=cat_features, eval_set=(X_val, y_val), early_stopping_rounds=50)

# Record training end time
end_time = time.time()

# Predict and evaluate RMSE
y_pred = model.predict(X_val)
rmse = mean_squared_error(y_val, y_pred, squared=False)
training_time = end_time - start_time

# Print results
print(f"\nValidation RMSE: {rmse:.4f}")
print(f"Training Time: {training_time:.2f} seconds")


0:	learn: 11.2960412	test: 11.2952806	best: 11.2952806 (0)	total: 2.67s	remaining: 44m 22s
100:	learn: 7.3677136	test: 7.1473837	best: 7.1473837 (100)	total: 2m 49s	remaining: 25m 11s
200:	learn: 7.2289867	test: 7.0067909	best: 7.0067909 (200)	total: 6m 37s	remaining: 26m 18s
300:	learn: 7.1091997	test: 6.8838113	best: 6.8838113 (300)	total: 10m 36s	remaining: 24m 37s
400:	learn: 7.0181003	test: 6.7900228	best: 6.7900228 (400)	total: 14m 53s	remaining: 22m 15s
500:	learn: 6.9354469	test: 6.7047329	best: 6.7047329 (500)	total: 18m 51s	remaining: 18m 47s
600:	learn: 6.8673567	test: 6.6349359	best: 6.6349359 (600)	total: 22m 42s	remaining: 15m 4s
700:	learn: 6.8084312	test: 6.5744102	best: 6.5744102 (700)	total: 26m 33s	remaining: 11m 19s
800:	learn: 6.7553340	test: 6.5197205	best: 6.5197205 (800)	total: 30m 28s	remaining: 7m 34s
900:	learn: 6.7003650	test: 6.4634342	best: 6.4634342 (900)	total: 34m 14s	remaining: 3m 45s
999:	learn: 6.6505010	test: 6.4124797	best: 6.4124797 (999)	total: 3

In [16]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from catboost import CatBoostRegressor
import time

# Target column
target_col = 'Lap_Time_Seconds'

# Drop rows with missing target
train_df = train_df.dropna(subset=[target_col])

# Treat ID-like numeric columns as categorical by converting them to string
id_like_cats = ['Rider_ID', 'Rider', 'Team', 'Bike']
for col in id_like_cats:
    if col in train_df.columns:
        train_df[col] = train_df[col].astype(str)

# Split features and target
X = train_df.drop(columns=[target_col])
y = train_df[target_col]

# Split into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Identify categorical features for CatBoost
cat_features = X.select_dtypes(include=['object', 'category']).columns.tolist()

# Fill missing values (CatBoost can handle missing values too, but filling ensures no surprises)
X_train = X_train.fillna(-999)
X_val = X_val.fillna(-999)

# Initialize CatBoost Regressor
model = CatBoostRegressor(
    iterations=1000,
    learning_rate=0.05,
    depth=6,
    eval_metric='RMSE',
    random_seed=42,
    verbose=100
)

# Record training start time
start_time = time.time()

# Train model
model.fit(X_train, y_train, cat_features=cat_features, eval_set=(X_val, y_val), early_stopping_rounds=50)

# Record training end time
end_time = time.time()

# Predict and evaluate RMSE
y_pred = model.predict(X_val)
rmse = mean_squared_error(y_val, y_pred, squared=False)
training_time = end_time - start_time

# Print results
print(f"\nValidation RMSE: {rmse:.4f}")
print(f"Training Time: {training_time:.2f} seconds")


0:	learn: 11.2211747	test: 11.2167099	best: 11.2167099 (0)	total: 5s	remaining: 1h 23m 17s
100:	learn: 7.3517969	test: 7.1276240	best: 7.1276240 (100)	total: 3m 56s	remaining: 35m 6s
200:	learn: 7.1995888	test: 6.9726537	best: 6.9726537 (200)	total: 9m 23s	remaining: 37m 20s
300:	learn: 7.0807866	test: 6.8497135	best: 6.8497135 (300)	total: 15m	remaining: 34m 52s
400:	learn: 6.9823707	test: 6.7482371	best: 6.7482371 (400)	total: 20m 34s	remaining: 30m 44s
500:	learn: 6.8956161	test: 6.6582851	best: 6.6582851 (500)	total: 25m 47s	remaining: 25m 41s
600:	learn: 6.8093905	test: 6.5682072	best: 6.5682072 (600)	total: 31m 9s	remaining: 20m 41s
700:	learn: 6.7359306	test: 6.4921946	best: 6.4921946 (700)	total: 36m 27s	remaining: 15m 33s
800:	learn: 6.6678359	test: 6.4212650	best: 6.4212650 (800)	total: 41m 30s	remaining: 10m 18s
900:	learn: 6.6153987	test: 6.3675843	best: 6.3675843 (900)	total: 46m 20s	remaining: 5m 5s
999:	learn: 6.5617621	test: 6.3120130	best: 6.3120130 (999)	total: 51m 11